# OSN Competition round 2

In [ ]:
cd /cluster/home/figu/OSN_competition/round2/delivery

In [ ]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from tqdm import tqdm
import numpy as np
import plotly.express as px
import itertools
import multiprocessing as mp
import plotly.graph_objects as go
import geopy
from plotly.subplots import make_subplots
from pandas.io.json import json_normalize
import json
from itertools import combinations
import geopy.distance
import pickle
from scipy.optimize import fsolve, root
from scipy.stats import iqr
import warnings
from scipy.interpolate import UnivariateSpline

warnings.filterwarnings("ignore")

# Data loading

In [ ]:
training = pd.read_csv('round2_competition.csv')
training.head()

## Basic filtering

In [ ]:
# bad_ids = []
# ids2remove = []
# thr = 1e4
# for ac_id in tqdm(list(training.aircraft.unique())):
#     tac = training.loc[training.aircraft==ac_id]
#     d = np.sqrt(np.diff(tac.latitude)**2+np.diff(tac.longitude)**2)*111000/np.diff(tac.timeAtServer).tolist()
#     if np.any(d>thr):
#         outliers_ids = tac.iloc[np.where(d>thr)].id.values.tolist()
#         ids2remove += outliers_ids
#         traj_out = tac.loc[tac.id.isin(outliers_ids)]
#         plt.figure()
#         plt.scatter(tac.longitude, tac.latitude)
#         plt.scatter(traj_out.longitude, traj_out.latitude, color='red')
#         plt.show()

In [ ]:
sample = pd.read_csv('round2_sample_empty.csv')
print(len(sample))
sample.head()

In [ ]:
sensors = pd.read_csv('round2_sensors.csv')
sensors.head()

In [ ]:
good_sensors = sensors.loc[sensors.good].serial.values

# Expand columns / rows

In [ ]:
def expand_measurements(df):
    # Function to expand the json coumn called measurements
    dfs = []
    def json_to_df(row, json_col):
        json_df = pd.read_json(row[json_col])
        dfs.append(json_df.assign(**row.drop(json_col)))
    df.apply(json_to_df, axis=1, json_col='measurements')   
    return pd.concat(dfs).reset_index()

def expand_measurements_para(df):
    # Here we used multoprocessing to make things faster
    num_processes = mp.cpu_count()
    chunk_size = int((len(df)//num_processes)+1)
    chunks = [df.iloc[i:i + chunk_size]for i in range(0, len(df), chunk_size)]

    with mp.Pool(num_processes) as pool:
        list_df = pool.map(expand_measurements, chunks)
        pool.close()
        pool.join()
    return pd.concat(list_df)


In [ ]:
try:
    df_training = pd.read_pickle('df_competition.pkl')
except:
    df_training = expand_measurements_para(training)
    df_training.to_pickle('df_competition.pkl')
df_training = df_training.rename(columns={0: 'sensor', 1:'timestamp', 2:'power'})
df_training.head()

In [ ]:
len(df_training.id.unique())

In [ ]:
df_testing = df_training.loc[df_training.latitude.isnull()]
df_training = df_training.loc[~df_training.latitude.isnull()]

# Pairwise and triplet wise
Here we extract data to have pairwise / triplet wise information

In [ ]:
def get_pairs_dt_n_power(df):
    rows_list = []
    grouped = df.groupby('id')
    for ids, group in tqdm(grouped, total=len(grouped)):
        sensor_pairs = [sorted(t) for t in list(combinations(group.sensor, 2))]
        for pair in sensor_pairs:
            dt_obs = group.loc[group.sensor==pair[0]].timestamp.values[0]-group.loc[group.sensor==pair[1]].timestamp.values[0]
            p0 = group.loc[group.sensor==pair[0]].power.values[0]
            p1 = group.loc[group.sensor==pair[1]].power.values[0]
            tAtServer = group.loc[group.sensor==pair[1]].timeAtServer.values[0]
            lat = group.latitude.values[0]
            lon = group.longitude.values[0]
            baro = group.baroAltitude.values[0]
            geo =  group.geoAltitude.values[0]
            rows_list.append({'id':ids, 's0':pair[0], 's1':pair[1], 'dt_obs':dt_obs,
                              'p0': p0, 'p1': p1, 'timeAtServer': tAtServer,
                              'latitude': lat, 'longitude': lon, 'baroAltitude': baro, 'geoAltitude': geo})
    print('Job done')
    return pd.DataFrame(rows_list)

def get_pairs_dt_n_power_para(df):
    # Here we use multoprocessing to make things faster
    num_processes = mp.cpu_count()
    grouped = df.groupby('id')
    list_groups = [g[1] for g in list(grouped)]
    chunk_size = int((len(list_groups)//num_processes)+1)                      
    chunks = [pd.concat(list_groups[i:i+chunk_size]) for i in range(0, len(list_groups), chunk_size)]
    print('Start // computation')
    with mp.Pool(num_processes) as pool:
        list_df = pool.map(get_pairs_dt_n_power, chunks)
        pool.close()
        pool.join()
    return pd.concat(list_df)

def get_triplet_dt_n_power(list_ids):
    rows_list = []
    for ids in tqdm(list_ids):
        group = df2.loc[df2.id==ids]
        if len(group)<3:
            continue
        sensor_pairs = [sorted(t) for t in list(combinations(group.sensor, 3))]
        for pair in sensor_pairs:
            dt_01 = group.loc[group.sensor==pair[0]].timestamp.values[0]-group.loc[group.sensor==pair[1]].timestamp.values[0]
            dt_02 = group.loc[group.sensor==pair[0]].timestamp.values[0]-group.loc[group.sensor==pair[2]].timestamp.values[0]
            dt_12 = group.loc[group.sensor==pair[1]].timestamp.values[0]-group.loc[group.sensor==pair[2]].timestamp.values[0]
            p0 = group.loc[group.sensor==pair[0]].power.values[0]
            p1 = group.loc[group.sensor==pair[1]].power.values[0]
            p2 = group.loc[group.sensor==pair[2]].power.values[0]
            tAtServer = group.loc[group.sensor==pair[1]].timeAtServer.values[0]
            lat = group.latitude.values[0]
            lon = group.longitude.values[0]
            baro = group.baroAltitude.values[0]
            geo =  group.geoAltitude.values[0]
            rows_list.append({'id':ids, 's0':pair[0], 's1':pair[1], 's2':pair[2],
                              'dt01':dt_01, 'dt02':dt_02, 'dt12':dt_12,
                              'p0': p0, 'p1': p1, 'p2':p2, 'timeAtServer': tAtServer,
                              'latitude': lat, 'longitude': lon, 'baroAltitude': baro, 'geoAltitude': geo})
    return pd.DataFrame(rows_list)

def get_triplet_dt_n_power_para(df):
    # Here we use multoprocessing to make things faster
    global df2
    df2 = df.copy()
    num_processes = mp.cpu_count()
    list_ids = df.id.unique()
    chunk_size = int((len(list_ids)//num_processes)+1)                      
    chunks = [list_ids[i:i + chunk_size]for i in range(0, len(list_ids), chunk_size)]
    print('Start // computation')
    with mp.Pool(num_processes) as pool:
        list_df = pool.map(get_triplet_dt_n_power, chunks)
        pool.close()
        pool.join()
    return pd.concat(list_df)


def get_triplet_dt_n_power2(list_ids):
    rows_list = []
    for ids in tqdm(list_ids):
        group = dico_groups2[ids]
        if len(group)<3:
            continue
        sensor_pairs = [sorted(t) for t in list(combinations(group.sensor, 3))]
        for pair in sensor_pairs:
            dt_01 = group.loc[group.sensor==pair[0]].timestamp.values[0]-group.loc[group.sensor==pair[1]].timestamp.values[0]
            dt_02 = group.loc[group.sensor==pair[0]].timestamp.values[0]-group.loc[group.sensor==pair[2]].timestamp.values[0]
            dt_12 = group.loc[group.sensor==pair[1]].timestamp.values[0]-group.loc[group.sensor==pair[2]].timestamp.values[0]
            p0 = group.loc[group.sensor==pair[0]].power.values[0]
            p1 = group.loc[group.sensor==pair[1]].power.values[0]
            p2 = group.loc[group.sensor==pair[2]].power.values[0]
            tAtServer = group.loc[group.sensor==pair[1]].timeAtServer.values[0]
            lat = group.latitude.values[0]
            lon = group.longitude.values[0]
            baro = group.baroAltitude.values[0]
            geo =  group.geoAltitude.values[0]
            rows_list.append({'id':ids, 's0':pair[0], 's1':pair[1], 's2':pair[2],
                              'dt01':dt_01, 'dt02':dt_02, 'dt12':dt_12,
                              'p0': p0, 'p1': p1, 'p2':p2, 'timeAtServer': tAtServer,
                              'latitude': lat, 'longitude': lon, 'baroAltitude': baro, 'geoAltitude': geo})
    return pd.DataFrame(rows_list)


def get_triplet_dt_n_power_para2(dico_groups):
    # Here we use multoprocessing to make things faster
    global dico_groups2
    dico_groups2 = dico_groups.copy()
    num_processes = mp.cpu_count()
    list_ids = list(dico_groups.keys())
    chunk_size = int((len(list_ids)//num_processes)+1)                      
    chunks = [list_ids[i:i + chunk_size]for i in range(0, len(list_ids), chunk_size)]
    print('Start // computation')
    with mp.Pool(num_processes) as pool:
        list_df = pool.map(get_triplet_dt_n_power2, chunks)
        pool.close()
        pool.join()
    return pd.concat(list_df)

### Pairwise information on the training dataset

This might take several hours to run (> 2hours on my machine)

In [ ]:
try:
    X_train_pairs = pd.read_pickle('X_train_pairs.pkl')
except:
    X_train_pairs = get_pairs_dt_n_power_para(df_training)
    X_train_pairs.to_pickle('X_train_pairs.pkl')
X_train_pairs.head()

### Tripletwise data for the training and testing dataset

In [ ]:
try:
    X_train_triplet = pd.read_pickle('X_comp_trip.pkl')
except:
    grouped = df_training.groupby('id')
    dico_groups = {ids: dft for ids, dft in tqdm(grouped)}
    dico_groups_sub = {ids:dico_groups[ids] for cpt, ids in enumerate(dico_groups) if cpt < 200000}
    X_train_triplet = get_triplet_dt_n_power_para2(dico_groups_sub)
    X_train_triplet.to_pickle('X_comp_trip.pkl')
X_train_triplet.head()

In [ ]:
try:
    X_test_triplet = pd.read_pickle('X_test_trip.pkl')
except:
    grouped = df_testing.groupby('id')
    dico_groups = {ids: dft for ids, dft in tqdm(grouped)}
    dico_groups_sub = {ids:dico_groups[ids] for cpt, ids in enumerate(dico_groups)}
    X_test_triplet = get_triplet_dt_n_power_para2(dico_groups_sub)
    X_test_triplet.to_pickle('X_test_trip.pkl')
X_test_triplet.head()

# Barometric altitude estimation using LGBM
## Put in in format for lgbm:

In [ ]:
X_train = X_train_triplet.copy()
X_test = X_test_triplet.copy()

In [ ]:
dico_sensors = {ids+1: tuple(sensors.iloc[ids][['latitude', 'longitude','height']].values) for ids in range(len(sensors))}

In [ ]:
X_train['s0_lla'] = [dico_sensors[x] for x in X_train.s0.values]
X_train['s1_lla'] = [dico_sensors[x] for x in X_train.s1.values]
X_train['s2_lla'] = [dico_sensors[x] for x in X_train.s2.values]
X_train[['s0_lat', 's0_lon', 's0_alt']] = pd.DataFrame(X_train['s0_lla'].tolist(), index=X_train.index)
X_train[['s1_lat', 's1_lon', 's1_alt']] = pd.DataFrame(X_train['s1_lla'].tolist(), index=X_train.index)
X_train[['s2_lat', 's2_lon', 's2_alt']] = pd.DataFrame(X_train['s2_lla'].tolist(), index=X_train.index)

X_train.id = X_train.id.astype(np.uint32)
X_train[['p0', 'p1', 'p2']] = X_train[['p0', 'p1', 'p2']].astype(np.uint8)
X_train[['latitude', 'longitude', 'baroAltitude', 'geoAltitude']] = X_train[['latitude', 'longitude', 'baroAltitude', 'geoAltitude']].astype(np.float32)

X_train["s0"] = X_train["s0"].astype('category')
X_train["s1"] = X_train["s1"].astype('category')
X_train["s2"] = X_train["s2"].astype('category')

X_train['s0s1s2'] = list(zip(X_train['s0'], X_train['s1'], X_train['s2']))

In [ ]:
X_test['s0_lla'] = [dico_sensors[x] for x in X_test.s0.values]
X_test['s1_lla'] = [dico_sensors[x] for x in X_test.s1.values]
X_test['s2_lla'] = [dico_sensors[x] for x in X_test.s2.values]
X_test[['s0_lat', 's0_lon', 's0_alt']] = pd.DataFrame(X_test['s0_lla'].tolist(), index=X_test.index)
X_test[['s1_lat', 's1_lon', 's1_alt']] = pd.DataFrame(X_test['s1_lla'].tolist(), index=X_test.index)
X_test[['s2_lat', 's2_lon', 's2_alt']] = pd.DataFrame(X_test['s2_lla'].tolist(), index=X_test.index)

X_test.id = X_test.id.astype(np.uint32)
X_test[['p0', 'p1', 'p2']] = X_test[['p0', 'p1', 'p2']].astype(np.uint8)
X_test[['latitude', 'longitude', 'baroAltitude', 'geoAltitude']] = X_test[['latitude', 'longitude', 'baroAltitude', 'geoAltitude']].astype(np.float32)


X_test["s0"] = X_test["s0"].astype('category')
X_test["s1"] = X_test["s1"].astype('category')
X_test["s2"] = X_test["s2"].astype('category')

X_test['s0s1s2'] = list(zip(X_test['s0'], X_test['s1'], X_test['s2']))

## Model Training

In [ ]:
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error

train_cols = ['p0', 'p1', 'p2', 'timeAtServer', 'baroAltitude',
              's0_lat', 's0_lon', 's0_alt', 's1_lat', 's1_lon', 's1_alt', 's2_lat', 's2_lon', 's2_alt']

y_col = ['geoAltitude']
X_training = X_train[train_cols]
Y = X_train[y_col]

# We will train 2 models each on one half of the data and sum it up at the end
X_training, x_valid, y_train, y_valid = train_test_split(X_training, Y, test_size=0.3, random_state=42)
d_train = lgb.Dataset(X_training, label=y_train)
d_valid = lgb.Dataset(x_valid, label=y_valid)
watchlist = [d_valid]

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'max_depth': 5,
    'num_leaves': 20,
    'learning_rate': 0.4,
    'verbose': 0, 
    'early_stopping_round': 5,
    }
n_estimators = 50


model_alt = lgb.train(params, d_train, n_estimators, watchlist, verbose_eval=1,
                       learning_rates=lambda iter: 0.01+0.8 * (0.99995 ** iter),
                      )


#### Test the model predicitons on a random sample

In [ ]:
from matplotlib import pyplot as plt
samp = X_train.sample(2000000)
plt.hist(model_alt.predict(samp[train_cols])-samp.geoAltitude, 100)
plt.hist(samp.baroAltitude-samp.geoAltitude, 100)
plt.show()

## Make predictions

In [ ]:
X_test.geoAltitude = model_alt.predict(X_test[train_cols])

# Sensor Time Synchronizations
For each pair of sensor, we will try to estimate the function (a polynome) that describe the sensors tdoa error:
$tdoa_{error} = tdoa_{observed}-tdoa_{computed}$. And then we try to find a function for each pair: $error(s0, s1, t1) = a0 + a1*t +... an*t^n)$

## Compute theoretical TDOA

In [ ]:
def get_calc_dt(r):
    try:
        d0 = geopy.distance.distance((r.s0_lat, r.s0_lon), (r.latitude, r.longitude)).m
        d0 = np.sqrt(d0**2 + (r.s0_alt-r.geoAltitude)**2)
        d1 = geopy.distance.distance((r.s1_lat, r.s1_lon), (r.latitude, r.longitude)).m
        d1 = np.sqrt(d1**2 + (r.s1_alt-r.geoAltitude)**2)
        dt01_calc = (d0 - d1)/0.2995
        return dt01_calc
    except:
        return np.nan

def get_calc_dt_df(df):
    df['dt_calc'] = df.apply(lambda r: get_calc_dt(r), 1)
    return df

def get_calc_dt_df_para(df):
    # Here we use multoprocessing to make things faster
    num_processes = mp.cpu_count()
    chunk_size = int((len(df)//num_processes)+1)                      
    chunks = [df.iloc[i:i + chunk_size]for i in range(0, len(df), chunk_size)]
    print('Start // computation')
    with mp.Pool(num_processes) as pool:
        list_df = pool.map(get_calc_dt_df, chunks)
        pool.close()
        pool.join()
    return pd.concat(list_df)

try:
    X_train_pairs = pd.read_pickle('X_train_pairs_dt_calc.pkl')
except:
    X_train_pairs['s0_lla'] = [dico_sensors[x] for x in X_train_pairs.s0.values]
    X_train_pairs['s1_lla'] = [dico_sensors[x] for x in X_train_pairs.s1.values]
    X_train_pairs[['s0_lat', 's0_lon', 's0_alt']] = pd.DataFrame(X_train_pairs['s0_lla'].tolist(), index=X_train_pairs.index)
    X_train_pairs[['s1_lat', 's1_lon', 's1_alt']] = pd.DataFrame(X_train_pairs['s1_lla'].tolist(), index=X_train_pairs.index)
    X_train_pairs = get_calc_dt_df_para(X_train_pairs)
    X_train_pairs.to_pickle('X_train_pairs_dt_calc.pkl')

#### We retrieve all the pairs that are in the testing set

In [ ]:
pairs = (X_test[['s0', 's1']].values.tolist() +
         X_test[['s0', 's2']].values.tolist() + 
         X_test[['s1', 's2']].values.tolist())
pairs = set([tuple(p) for p in pairs])
print(len(pairs))

In [ ]:
X_train_pairs

In [ ]:
X_train_pairs = X_train_pairs.dropna(subset=['dt_calc'])
print(len(X_train_pairs))

### For all pairs we compute the polynom that improve the fit:

In [ ]:
def is_outlier(points, thresh=3):
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh

def get_dico_corrections(pairs):
    score_rng = [15, 85]
    dico_poly_correction = {}
    pairs = list(pairs)
    for id0, id1 in tqdm(pairs):
        t_test = X_test.loc[((X_test.s0==id0) & (X_test.s1==id1))
                           | ((X_test.s1==id0) & (X_test.s2==id1) )
                           | ((X_test.s0==id0) & (X_test.s2==id1))].timeAtServer
        # First we retrieve the theoretical tdoa
        sub_pair = X_train_pairs.loc[(X_train_pairs.s0==id0) & (X_train_pairs.s1==id1)]
        if len(sub_pair)<2:
            continue
        sub_pair['err'] = sub_pair['dt_obs']-sub_pair['dt_calc']
        # then we fist a 1st order to eliminate outliers
        poly = np.poly1d(np.polyfit(sub_pair['timeAtServer'], sub_pair['err'], 1))
        sub_pair['diff'] = sub_pair['err']-poly(sub_pair.timeAtServer)
        sub_pair = sub_pair.loc[~is_outlier(sub_pair['diff'], 3.5)]
        #############
        poly = np.poly1d(np.polyfit(sub_pair['timeAtServer'], sub_pair['err'], 1))
        sub_pair['diff'] = sub_pair['err']-poly(sub_pair.timeAtServer)
        sub_pair = sub_pair.loc[~is_outlier(sub_pair['diff'], 3.5)]
        ######################

        # then we refit a 1st  and a 30st order on the filtered version
        poly_best = poly1 = np.poly1d(np.polyfit(sub_pair['timeAtServer'], sub_pair['err'], 1))
        diff_best = sub_pair['err']-poly_best(sub_pair.timeAtServer)
        smin = iqr(diff_best, rng=score_rng)
        lsp = len(sub_pair)
        if lsp > 10:
            for i in range(1, np.min([int(lsp/10), 30])):
                poly = np.poly1d(np.polyfit(sub_pair['timeAtServer'], sub_pair['err'], i))
                diff = sub_pair['err']-poly(sub_pair.timeAtServer)
                score = iqr(diff, rng=score_rng)
                if score < smin:
                    smin = score
                    poly_best = poly
        
        diff1 = sub_pair['err']-poly1(sub_pair.timeAtServer)
        diff_best = sub_pair['err']-poly_best(sub_pair.timeAtServer)
        dico_poly_correction[id0, id1] = (poly_best,
                                          iqr(diff_best, rng=score_rng),
                                          sub_pair['timeAtServer'].values,
                                          poly1,
                                          iqr(diff1, rng=score_rng))
    return dico_poly_correction

try:
    with open('dico_poly_correction.pickle', 'rb') as handle:
        dico_poly_correction = pickle.load(handle)
except:
    dico_poly_correction =  get_dico_corrections(pairs)
    with open('dico_poly_correction.pickle', 'wb') as handle:
        pickle.dump(dico_poly_correction, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Functions to correct observed tdoa

In [ ]:
def correct_tdoa01(row, dico_poly_correction):
    max_val = 2000
    if (row.s0, row.s1) in dico_poly_correction:
        if np.min(dico_poly_correction[(row.s0, row.s1)][2]) < row.timeAtServer < np.max(dico_poly_correction[(row.s0, row.s1)][2]):
            if dico_poly_correction[(row.s0, row.s1)][1]<max_val:
                return row.dt01-dico_poly_correction[(row.s0, row.s1)][0](row.timeAtServer)
        elif dico_poly_correction[(row.s0, row.s1)][4]<max_val:
            return row.dt01-dico_poly_correction[(row.s0, row.s1)][3](row.timeAtServer)
    return np.nan
def correct_tdoa02(row, dico_poly_correction):
    max_val = 2000
    if (row.s0, row.s2) in dico_poly_correction:
        if np.min(dico_poly_correction[(row.s0, row.s2)][2]) < row.timeAtServer < np.max(dico_poly_correction[(row.s0, row.s2)][2]):
            if dico_poly_correction[(row.s0, row.s2)][1]<max_val:
                return row.dt02-dico_poly_correction[(row.s0, row.s2)][0](row.timeAtServer)
        elif dico_poly_correction[(row.s0, row.s2)][4]<max_val:
            return row.dt02-dico_poly_correction[(row.s0, row.s2)][3](row.timeAtServer)
    return np.nan
def correct_tdoa12(row, dico_poly_correction):
    max_val = 2000
    if (row.s1, row.s2) in dico_poly_correction:
        if np.min(dico_poly_correction[(row.s1, row.s2)][2]) < row.timeAtServer < np.max(dico_poly_correction[(row.s1, row.s2)][2]):
            if dico_poly_correction[(row.s1, row.s2)][1]<max_val:
                return row.dt12-dico_poly_correction[(row.s1, row.s2)][0](row.timeAtServer)
        elif dico_poly_correction[(row.s1, row.s2)][4]<max_val:
            return row.dt12-dico_poly_correction[(row.s1, row.s2)][3](row.timeAtServer)
    return np.nan

def add_tdoa_confidence(row, dico_poly_correction):
    score = 0
    if (row.s0, row.s1) in dico_poly_correction:
        score += dico_poly_correction[(row.s0, row.s1)][1]
    else:
        return np.nan
    if (row.s0, row.s2) in dico_poly_correction:
        score += dico_poly_correction[(row.s0, row.s2)][1]
    else:
        return np.nan
    if (row.s1, row.s2) in dico_poly_correction:
        score += dico_poly_correction[(row.s1, row.s2)][1]
    else:
        return np.nan
    return score



def correct_tdoa01(row, dico_poly_correction):
    max_val = 3000
    if (row.s0, row.s1) in dico_poly_correction:
        if np.min(dico_poly_correction[(row.s0, row.s1)][2]) < row.timeAtServer < np.max(dico_poly_correction[(row.s0, row.s1)][2]):
            if dico_poly_correction[(row.s0, row.s1)][1]<max_val:
                return row.dt01-dico_poly_correction[(row.s0, row.s1)][0](row.timeAtServer)
        elif dico_poly_correction[(row.s0, row.s1)][4]<max_val:
            return row.dt01-dico_poly_correction[(row.s0, row.s1)][3](row.timeAtServer)
    return np.nan
def correct_tdoa02(row, dico_poly_correction):
    max_val = 3000
    if (row.s0, row.s2) in dico_poly_correction:
        if np.min(dico_poly_correction[(row.s0, row.s2)][2]) < row.timeAtServer < np.max(dico_poly_correction[(row.s0, row.s2)][2]):
            if dico_poly_correction[(row.s0, row.s2)][1]<max_val:
                return row.dt02-dico_poly_correction[(row.s0, row.s2)][0](row.timeAtServer)
        elif dico_poly_correction[(row.s0, row.s2)][4]<max_val:
            return row.dt02-dico_poly_correction[(row.s0, row.s2)][3](row.timeAtServer)
    return np.nan
def correct_tdoa12(row, dico_poly_correction):
    max_val = 3000
    if (row.s1, row.s2) in dico_poly_correction:
        if np.min(dico_poly_correction[(row.s1, row.s2)][2]) < row.timeAtServer < np.max(dico_poly_correction[(row.s1, row.s2)][2]):
            if dico_poly_correction[(row.s1, row.s2)][1]<max_val:
                return row.dt12-dico_poly_correction[(row.s1, row.s2)][0](row.timeAtServer)
        elif dico_poly_correction[(row.s1, row.s2)][4]<max_val:
            return row.dt12-dico_poly_correction[(row.s1, row.s2)][3](row.timeAtServer)
    return np.nan

def add_tdoa_confidence(row, dico_poly_correction):
    score = 0
    if (row.s0, row.s1) in dico_poly_correction:
        score += dico_poly_correction[(row.s0, row.s1)][1]
    else:
        return np.nan
    if (row.s0, row.s2) in dico_poly_correction:
        score += dico_poly_correction[(row.s0, row.s2)][1]
    else:
        return np.nan
    if (row.s1, row.s2) in dico_poly_correction:
        score += dico_poly_correction[(row.s1, row.s2)][1]
    else:
        return np.nan
    return score

# Correction V2

In [ ]:
def get_cor_v2_int(dico_poly_correction, id1, id2, thr, t, rdtxx):
    offsets = []
#     Case intermediary sensor in middle id1 < k[1] < id2
    good_keys = [k  for k in dico_poly_correction if ((k[0]==id1) 
                 and (dico_poly_correction[k][2][0] <= t <= dico_poly_correction[k][2][-1])
                 and ((k[1], id2) in dico_poly_correction) 
                 and  (dico_poly_correction[k][1]<thr)
                 and  (dico_poly_correction[(k[1], id2)][1]<thr))]
    for k in good_keys:
        offset = rdtxx - (dico_poly_correction[k][0](t) + dico_poly_correction[k[1], id2][0](t))
#         if np.abs(offset) < 1E7:
        offsets.append(offset)
    
    # Case intermediary sensor right id1 < id2 < k[1]
    good_keys = [k for k in dico_poly_correction if ((k[0]==id1) 
                 and (dico_poly_correction[k][2][0] <= t <= dico_poly_correction[k][2][-1])
                 and ((id2, k[1]) in dico_poly_correction) 
                 and  (dico_poly_correction[k][1]<thr)
                 and (dico_poly_correction[(id2, k[1])][1]<thr))]
    for k in good_keys:
        offset = rdtxx - (dico_poly_correction[k][0](t) - dico_poly_correction[id2, k[1]][0](t))
#         if np.abs(offset) < 1E7:
        offsets.append(offset)

    # Case intermediary sensor left k[0] < id1 < id2
#     good_keys = [k for k in dico_poly_correction if ((k[1]==id1) 
#                  and (dico_poly_correction[k][2][0] <= t <= dico_poly_correction[k][2][-1])
#                  and ((k[0], id2) in dico_poly_correction) 
#                  and (dico_poly_correction[k][1]<thr)
#                  and (dico_poly_correction[(k[0], id2)][1]<thr))]
#     for k in good_keys:
#         offset = rdtxx - (-dico_poly_correction[k][0](t) + dico_poly_correction[k[1], id2][0](t))
#         if np.abs(offset) < 1E7:
#             offsets.append(offset)

    # Direct Pair case
#     if (id1, id2) in dico_poly_correction:
#         if (dico_poly_correction[(id1, id2)][2][0] <= t <= dico_poly_correction[(id1, id2)][2][-1]) \
#         and (dico_poly_correction[(id1, id2)][1] < thr):
#             offset = rdtxx-dico_poly_correction[id1, id2][0](t)
#             if np.abs(offset) < 1E7:
#                 offsets.append(offset)
            
    return offsets


def get_cor_v2_final(X_test_filtered_ac, dico_poly_correction):
    thr = 1600
    sol_01 = []
    sol_02 = []
    sol_12 = []
    for _, r in tqdm(X_test_filtered_ac.iterrows(), total=len(X_test_filtered_ac)):
        t = r.timeAtServer
        id1, id2 = r.s0, r.s1
        sol_01.append(get_cor_v2_int(dico_poly_correction, r.s0, r.s1, thr, t, r['dt01']))
        sol_02.append(get_cor_v2_int(dico_poly_correction, r.s0, r.s2, thr, t, r['dt02']))
        sol_12.append(get_cor_v2_int(dico_poly_correction, r.s1, r.s2, thr, t, r['dt12']))
    X_test_filtered_ac['dt01_cor_v2'] = sol_01
    X_test_filtered_ac['dt12_cor_v2'] = sol_12
    X_test_filtered_ac['dt02_cor_v2'] = sol_02
    X_test_filtered_ac['dt01_cor_v2_med'] = X_test_filtered_ac['dt01_cor_v2'].apply(lambda x: np.median(x))
    X_test_filtered_ac['dt01_cor_v2_n'] = X_test_filtered_ac['dt01_cor_v2'].apply(lambda x: len(x))
    X_test_filtered_ac['dt01_cor_v2_iqr'] = X_test_filtered_ac['dt01_cor_v2'].apply(lambda x: iqr(x))

    X_test_filtered_ac['dt02_cor_v2_med'] = X_test_filtered_ac['dt02_cor_v2'].apply(lambda x: np.median(x))
    X_test_filtered_ac['dt02_cor_v2_n'] = X_test_filtered_ac['dt02_cor_v2'].apply(lambda x: len(x))
    X_test_filtered_ac['dt02_cor_v2_iqr'] = X_test_filtered_ac['dt02_cor_v2'].apply(lambda x: iqr(x))

    X_test_filtered_ac['dt12_cor_v2_med'] = X_test_filtered_ac['dt12_cor_v2'].apply(lambda x: np.median(x))
    X_test_filtered_ac['dt12_cor_v2_n'] = X_test_filtered_ac['dt12_cor_v2'].apply(lambda x: len(x))
    X_test_filtered_ac['dt12_cor_v2_iqr'] = X_test_filtered_ac['dt12_cor_v2'].apply(lambda x: iqr(x))

    # Here we try to give a score the lower the better
#     X_test_filtered_ac['score'] = (X_test_filtered_ac['dt01_cor_v2_iqr']/(0.5*X_test_filtered_ac['dt01_cor_v2_n']) +
#         X_test_filtered_ac['dt02_cor_v2_iqr']/(0.5*X_test_filtered_ac['dt02_cor_v2_n']) +
#         X_test_filtered_ac['dt12_cor_v2_iqr']/(0.5*X_test_filtered_ac['dt12_cor_v2_n']))
    
    X_test_filtered_ac.loc[(np.isnan(X_test_filtered_ac.dt01_cor)) & (X_test_filtered_ac.dt01_cor_v2_iqr<1000) & (X_test_filtered_ac.dt01_cor_v2_n>3), 'dt01_cor'] = X_test_filtered_ac.loc[(np.isnan(X_test_filtered_ac.dt01_cor)) & (X_test_filtered_ac.dt01_cor_v2_iqr<1000) & (X_test_filtered_ac.dt01_cor_v2_n>3), 'dt01_cor_v2_med']
    X_test_filtered_ac.loc[(np.isnan(X_test_filtered_ac.dt02_cor)) & (X_test_filtered_ac.dt02_cor_v2_iqr<1000) & (X_test_filtered_ac.dt02_cor_v2_n>3), 'dt02_cor'] = X_test_filtered_ac.loc[(np.isnan(X_test_filtered_ac.dt02_cor)) & (X_test_filtered_ac.dt02_cor_v2_iqr<1000) & (X_test_filtered_ac.dt02_cor_v2_n>3), 'dt02_cor_v2_med']
    X_test_filtered_ac.loc[(np.isnan(X_test_filtered_ac.dt12_cor)) & (X_test_filtered_ac.dt12_cor_v2_iqr<1000) & (X_test_filtered_ac.dt12_cor_v2_n>3), 'dt12_cor'] = X_test_filtered_ac.loc[(np.isnan(X_test_filtered_ac.dt12_cor)) & (X_test_filtered_ac.dt12_cor_v2_iqr<1000) & (X_test_filtered_ac.dt12_cor_v2_n>3), 'dt12_cor_v2_med']
    return X_test_filtered_ac


# Multilateration

In [ ]:
import os.path
import re

from os import listdir
from os.path import isfile, join
from tqdm import tqdm

def MLAT3(p, *data):
    eqs = []
    dico_eq = {}
    for _, group in data[0].iterrows():
        id0, id1, id2 = group.s0, group.s1, group.s2
        c = np.float64(0.2995) # Transmission speed
        x, y = p
        dt01 = group.dt01_cor
        dt02 = group.dt02_cor
        dt12 = group.dt12_cor
        s0 = tuple(group[['s0_lat', 's0_lon']].values)
        s1 = tuple(group[['s1_lat', 's1_lon']].values)
        s2 = tuple(group[['s2_lat', 's2_lon']].values)
        try:
            ap_alt = np.float64(group.geoAltitude)
        except:
            ap_alt = np.float64(group.baroAltitude)
        vert0 = ap_alt-group.s0_alt
        vert1 = ap_alt-group.s1_alt
        vert2 = ap_alt-group.s2_alt
        
        if ((id0, id1)) not in dico_eq:
            dico_eq[(id0, id1)] = 0
            eqs.append(
                (np.sqrt(geopy.distance.distance(s0, (x, y)).m**2 + vert0**2)
                 -np.sqrt(geopy.distance.distance(s1, (x, y)).m**2 + vert1**2))/c-dt01,)
        if ((id0, id2)) not in dico_eq:
            dico_eq[(id0, id2)] = 0
            eqs.append(
                (np.sqrt(geopy.distance.distance(s0, (x, y)).m**2 + vert0**2)
                -np.sqrt(geopy.distance.distance(s2, (x, y)).m**2 + vert2**2))/c-dt02,)        
        if ((id1, id2)) not in dico_eq:
            dico_eq[(id1, id2)] = 0
            eqs.append(
                (np.sqrt(geopy.distance.distance(s1, (x, y)).m**2 + vert1**2)
                 -np.sqrt(geopy.distance.distance(s2, (x, y)).m**2 + vert2**2))/c-dt12,) 
    return eqs




dico_res_lon = {}
dico_res_lat = {}
list_dico_sol = []


def get_sol_mlat_all_in_view(list_ac_ids):
    sol_path = 'pickled_sol'
    df_sol = pd.DataFrame()
    for ac_id in list_ac_ids:
        if os.path.isfile(sol_path+'/df_sol_mlat_{}.pkl'.format(ac_id)):
            continue
        list_dico_sol = []
        X_test_filtered_ac = X_test_filtered.loc[
            X_test_filtered.id.isin(df_testing.loc[df_testing.aircraft==ac_id].id.unique())]
        X_test_filtered_ac.geoAltitude = X_test_filtered_ac.baroAltitude+np.median(X_test_filtered_ac.geoAltitude-X_test_filtered_ac.baroAltitude)

        if len(X_test_filtered_ac)==0:
            df_sol = pd.DataFrame()
            df_sol.to_pickle(sol_path+'/df_sol_mlat_{}.pkl'.format(ac_id))
            continue
            
        # correct dt_obs
        if True:
            X_test_filtered_ac['dt01_cor'] = X_test_filtered_ac.apply(lambda r: correct_tdoa01(r, dico_poly_correction), 1)
            X_test_filtered_ac['dt02_cor'] = X_test_filtered_ac.apply(lambda r: correct_tdoa02(r, dico_poly_correction), 1)
            X_test_filtered_ac['dt12_cor'] = X_test_filtered_ac.apply(lambda r: correct_tdoa12(r, dico_poly_correction), 1)
            X_test_filtered_ac = get_cor_v2_final(X_test_filtered_ac, dico_poly_correction)
            X_test_filtered_ac = X_test_filtered_ac.dropna(subset=['dt01_cor', 'dt02_cor', 'dt12_cor'])
            X_test_filtered_ac['tdoa_conf'] = X_test_filtered_ac.apply(lambda r: add_tdoa_confidence(r, dico_poly_correction), 1)
        
        else:
            df_sol.to_pickle(sol_path+'/df_sol_mlat_{}.pkl'.format(ac_id))
            continue
        for ids, group in tqdm(X_test_filtered_ac.groupby('id'), total=len(X_test_filtered_ac.id.unique())):
            if True:
                centroid = np.median(group[['s0_lat', 's1_lat', 's2_lat']].values.flatten()), np.median(group[['s0_lon', 's1_lon', 's2_lon']].values.flatten())
                group = group.loc[(np.abs(group.dt01_cor) < 1e7) &
                                  (np.abs(group.dt02_cor) < 1e7) &
                                  (np.abs(group.dt12_cor) < 1e7)]
                group = group.loc[group.tdoa_conf<9000]
#                 group = group.sort_values('score')
                group = group.sort_values('tdoa_conf')

                group = group.iloc[:10]
                sample = group.iloc[0]
                guess = centroid
                
                # Bad first guess based on visual inspection
                if ac_id == 425:
                    guess = 40.05, -6.75
                if ac_id == 1515:
                    guess = 44.88, 15.11

                if len(list_dico_sol) > 0:
                    guess = list_dico_sol[-1]['lat'], list_dico_sol[-1]['lon']
                
                solution = root(MLAT3, guess, args=group, method='lm')
                if solution.status not in [1, 2, 3]:
                    continue
                list_dico_sol.append({'ID': ids,
                      'timeAtServer': sample.timeAtServer,
                      'guess': guess,
                      'lat': solution.x[0],
                      'lon': (solution.x[1] % 360 + 540) % 360 - 180,
                      'centroid_lat': centroid[0],
                      'centroid_lon': centroid[1],
                      'n':len(group),
                      'mean_fval': np.mean(solution.fun),
                      'qtf': solution.qtf,
                      'iqr_fval': iqr(solution.fun)
                     }
                    )
            else:
                pass
        df_sol = pd.DataFrame(list_dico_sol)
        df_sol.to_pickle(sol_path+'/df_sol_mlat_{}.pkl'.format(ac_id))

def get_sol_mlat_all_in_view_para(X_test_filtered2, df_testing2, sensors2):
    # Here we use multoprocessing to make things faster
    global X_test_filtered
    global df_testing
    global sensors

    X_test_filtered = X_test_filtered2.copy()
    df_testing = df_testing2.copy()
    sensors = sensors2.copy()
    
    dir_path = 'pickled_sol'
    onlyfiles = [f for f in listdir(dir_path) if isfile(join(dir_path, f))]
    str_pkl_files = ''.join(onlyfiles)
    num_processes = mp.cpu_count()
    temp = re.findall(r'\d+', str_pkl_files) 
    ids_done = list(map(int, temp)) 
    
    list_ids = list(set(list(df_testing.aircraft.unique()))-set(ids_done))
    print('IDs to be computed:', len(list_ids))
    if len(list_ids)==0:
        return
    chunk_size = int((len(list_ids)//num_processes)+1)                      
    chunks = [list_ids[i:i + chunk_size]for i in range(0, len(list_ids), chunk_size)]
    print('Start // computation')
    with mp.Pool(num_processes) as pool:
        list_df = pool.map(get_sol_mlat_all_in_view, chunks)
        pool.close()
        pool.join()
    return pd.concat(list_df)

In [ ]:
X_test_filtered = X_test.copy()
get_sol_mlat_all_in_view_para(X_test, df_testing, sensors)

# Results

In [ ]:
from matplotlib import pyplot as plt
from scipy.signal import savgol_filter
path_pkl = 'pickled_sol'

df_sols = []
df_sols3 = []
onlyfiles = [f for f in listdir(path_pkl) if isfile(join(path_pkl, f))]

def is_outlier(points, thresh=3):
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh

def filter_results(selection, var, aircraft, n_poly=3, delta_t=20):
    tas = testing[testing.aircraft==aircraft]
    selection['gt'] = selection.timeAtServer.apply(lambda x: (x-selection.timeAtServer.values[0])//delta_t).astype(int)
    x = selection.timeAtServer.values
    y = selection[var].values
    dico_tp = {}

    gtimes = {}
    for gt, group in selection.groupby('gt'):
        n_poly = np.min([len(group)//5,  n_poly])+1
        dico_tp[gt] = np.poly1d(np.polyfit(group.timeAtServer, group[var], n_poly))
        gtimes[gt] = group.timeAtServer.values.tolist()
    var2 = []
    for gt in dico_tp:
        var2 += dico_tp[gt](gtimes[gt]).tolist()
    selection['var2'] = var2
    selection['var_err'] = (selection.var2-selection[var]).abs()


    selection = selection.loc[~is_outlier(selection['var_err'], 3.5)]

    x = selection.timeAtServer.values
    y = selection[var].values
    gtimes = {}
    dico_tp = {}

    for gt, group in selection.groupby('gt'):
        n_poly = np.min([len(group)//5,  n_poly])+1
        dico_tp[gt] = np.poly1d(np.polyfit(group.timeAtServer, group[var], n_poly))
        gtimes[gt] = group.timeAtServer.values.tolist()
    var2 = []
    new_times = []
    ids = []
    for gt in dico_tp:
        if len(gtimes[gt])<2:
            continue
        inter_df = get_time_inter(gtimes[gt], tas)
        t_inter = inter_df.timeAtServer.values.tolist()
        var2 += dico_tp[gt](t_inter).tolist()
        ids += inter_df.id.values.tolist()
        new_times += t_inter
    df = pd.DataFrame()
    df['timeAtServer'] = new_times
    df[var] = var2
    df['id'] = ids
    return df

def get_time_inter(gt, tas):
    return tas[(gt[0]<=tas.timeAtServer) & (tas.timeAtServer<=gt[-1])]

def process_results(onlyfiles, n_min=1, qtf_max=1, delta_t=30, n_poly=3, plt_en=True):
    for cpt, f in enumerate(onlyfiles[:]):
        temp = re.findall(r'\d+', f) 
        ids = list(map(int, temp))[0]

        path_pkl = 'pickled_sol'
        
        tac = testing.loc[testing.aircraft==ids]
        mlat = pd.read_pickle(join(path_pkl, 'df_sol_mlat_{}.pkl'.format(ids) ))
        if len(mlat) > 0 :
            if len(set(mlat.ID) & set(tac.id)) < len(set(mlat.ID)):
                continue
#                 print('Achtung', ids, len(mlat), len(set(mlat.ID) & set(tac.id)))
                
        if ids in [357, 1570, 138, 1913, 2820, 1832, 2910, 1677, 847, 994, 1228, 1569, 1568, 182, 1887, 2144, 2459, 2744]:
            continue
        if plt_en:
            
            fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(19, 3))
            try:
                lb = last_best.loc[last_best.id.isin(testing.loc[testing.aircraft==ids].id)]
                ax1.scatter(x=lb['timeAtServer'], y=lb['latitude'], c='orange')
                ax2.scatter(x=lb['timeAtServer'], y=lb['longitude'], c='orange')
                ax3.scatter(x=lb['longitude'], y=lb['latitude'], c='orange')
                plt.title(str(ids))
            except:
                pass

        df_sol = pd.read_pickle(path_pkl+'/'+'df_sol_mlat_{}.pkl'.format(ids))
        if len(df_sol)<1:
            continue
        df_sol.qtf = df_sol.qtf.apply(lambda x: np.abs(x[0])+np.abs(x[1]))

        selection = df_sol.loc[(df_sol.n >= n_min) & 
                               (df_sol.qtf <= qtf_max) & 
                               (np.abs(df_sol.lat-df_sol.centroid_lat)<3) &
                               (np.abs(df_sol.lon-df_sol.centroid_lon)<3)
                              ].reset_index()

        if plt_en:
#             pass
            ax1.scatter(x=selection['timeAtServer'], y=selection['lat'], c='blue', s=10)
            ax2.scatter(x=selection['timeAtServer'], y=selection['lon'], c='blue', s=10)
#             ax3.scatter(x=selection['lon'], y=selection['lat'], c='blue', s=10)

        samp = sample.loc[sample.id.isin(testing[testing.aircraft==ids].id)]
    #     break
        samp = samp.set_index('id').join(selection[['lat', 'lon', 'ID']].set_index('ID'), rsuffix='_')
        samp.latitude.fillna(samp.lat, inplace=True)
        samp.longitude.fillna(samp.lon, inplace=True)
        samp = samp.dropna(subset=['latitude'])
        samp = samp.dropna(subset=['longitude'])
        selection = samp.drop(['lat', 'lon'], axis=1)
        selection = selection.rename(columns={'latitude':'lat', 'longitude':'lon'})

        if plt_en:
            ax1.scatter(x=selection['timeAtServer'], y=selection['lat'], c='black', s=10)
            ax2.scatter(x=selection['timeAtServer'], y=selection['lon'], c='black', s=10)
#             ax3.scatter(x=selection['lon'], y=selection['lat'], c='blue', s=10)
            ax2.set_title(str(ids))

        filtered_selection_lon = filter_results(selection.dropna(subset=['lon']), 'lon', ids, n_poly=n_poly, delta_t=delta_t)
        filtered_selection_lat = filter_results(selection.dropna(subset=['lat']), 'lat', ids, n_poly=n_poly, delta_t=delta_t)
        sample.loc[sample.id.isin(filtered_selection_lon.id.values), 'longitude'] = filtered_selection_lon.lon.values
        sample.loc[sample.id.isin(filtered_selection_lat.id.values), 'latitude'] = filtered_selection_lat.lat.values

        samp = sample.loc[sample.id.isin(testing[testing.aircraft==ids].id)]
#         samp_ac = sample.loc[sample.id.isin(filtered_selection_lon.id.values)]
        if plt_en:
            print(ids)
            ax1.scatter(x=samp['timeAtServer'], y=samp['latitude'], c='red', s=1, zorder=99999)
            ax2.scatter(x=samp['timeAtServer'], y=samp['longitude'], c='red', s=1, zorder=99999)
            ax3.scatter(x=samp['longitude'], y=samp['latitude'], c='red', s=1)
            print(100*(len(samp.dropna(subset=['latitude']).dropna(subset=['longitude']))/len(sample)))
            ax1.set_title(str(ids))
            plt.show()
    return sample

 

In [ ]:
testing = training.loc[training.latitude.isnull()]
testing

In [ ]:
sample = pd.read_csv('round2_sample_empty.csv')
print(len(sample))
sample = sample.set_index('id').join(testing.set_index('id')['timeAtServer']).reset_index()
sample.head()

In [ ]:
# BEst one
n = 300
sample = process_results(onlyfiles[:n], n_min=3, qtf_max=1e-3, delta_t=30, n_poly=3, plt_en=False)
sample = process_results(onlyfiles[:n], n_min=3, qtf_max=1e-3, delta_t=70, n_poly=3, plt_en=False)

print('done')
sample = process_results(onlyfiles[:n], n_min=2, qtf_max=1e-1, delta_t=70, n_poly=4, plt_en=False)
sample = process_results(onlyfiles[:n], n_min=1, qtf_max=1, delta_t=100, n_poly=4, plt_en=False)
sample = process_results(onlyfiles[:n], n_min=1, qtf_max=2, delta_t=200, n_poly=4, plt_en=True)


In [ ]:
perc = 100*(len(sample.dropna(subset=['latitude']).dropna(subset=['longitude']))/len(sample))
print(perc)
if perc >= 70:
    sample = sample[['id', 'latitude','longitude','geoAltitude']]
    sample.to_csv('sol_osn_comp.csv', index=False)